#Momento de Retroalimentación: Módulo 1 Utilización, procesamiento y visualización de grandes volúmenes de datos (Portafolio Análisis)

Para esta actividad, se hara la prediccion sobre si una persona es propensa a tener un ataque cardiaco segun sus datos medicos

##Inicio de sesion en spark

In [1]:
#Bibliotecas para poder trabajar con Spark
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.2//spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz
#Configuración de Spark con Python
!pip install -q findspark
!pip install pyspark

#Estableciendo variable de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

#Buscando e inicializando la instalación de Spark
import findspark
findspark.init()
findspark.find()

#Probando PySparl
from pyspark.sql import SparkSession

spark_Act1 = SparkSession \
       .builder \
       .appName("Act1") \
       .getOrCreate()

spark_Act1

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:13 https://developer.downlo

## Acceso a datos de Google Drive

In [3]:
from google.colab import drive

drive.mount("/content/gdrive") 

Mounted at /content/gdrive


In [4]:
%cd "/content/gdrive/MyDrive/Bloque 2 IA/Modulo 1 BD/Entregable"
!ls

/content/gdrive/MyDrive/Bloque 2 IA/Modulo 1 BD/Entregable
 heart.csv  'ML Pyspark.ipynb'


##Obtencion del dataset 

Dataset obtenido de kaggle [Heart Attack Analysis](https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset?select=heart.csv)

In [5]:
df = spark_Act1.read.csv('/content/gdrive/MyDrive/Bloque 2 IA/Modulo 1 BD/Entregable/heart.csv', header=True, inferSchema=True)
df

DataFrame[age: int, sex: int, cp: int, trtbps: int, chol: int, fbs: int, restecg: int, thalachh: int, exng: int, oldpeak: double, slp: int, caa: int, thall: int, output: int]

In [6]:
#Visualizamos el dataset para conocer los datos a tratar
df.show()

+---+---+---+------+----+---+-------+--------+----+-------+---+---+-----+------+
|age|sex| cp|trtbps|chol|fbs|restecg|thalachh|exng|oldpeak|slp|caa|thall|output|
+---+---+---+------+----+---+-------+--------+----+-------+---+---+-----+------+
| 63|  1|  3|   145| 233|  1|      0|     150|   0|    2.3|  0|  0|    1|     1|
| 37|  1|  2|   130| 250|  0|      1|     187|   0|    3.5|  0|  0|    2|     1|
| 41|  0|  1|   130| 204|  0|      0|     172|   0|    1.4|  2|  0|    2|     1|
| 56|  1|  1|   120| 236|  0|      1|     178|   0|    0.8|  2|  0|    2|     1|
| 57|  0|  0|   120| 354|  0|      1|     163|   1|    0.6|  2|  0|    2|     1|
| 57|  1|  0|   140| 192|  0|      1|     148|   0|    0.4|  1|  0|    1|     1|
| 56|  0|  1|   140| 294|  0|      0|     153|   0|    1.3|  1|  0|    2|     1|
| 44|  1|  1|   120| 263|  0|      1|     173|   0|    0.0|  2|  0|    3|     1|
| 52|  1|  2|   172| 199|  1|      1|     162|   0|    0.5|  2|  0|    3|     1|
| 57|  1|  2|   150| 168|  0

In [7]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- cp: integer (nullable = true)
 |-- trtbps: integer (nullable = true)
 |-- chol: integer (nullable = true)
 |-- fbs: integer (nullable = true)
 |-- restecg: integer (nullable = true)
 |-- thalachh: integer (nullable = true)
 |-- exng: integer (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slp: integer (nullable = true)
 |-- caa: integer (nullable = true)
 |-- thall: integer (nullable = true)
 |-- output: integer (nullable = true)



## Data preparation

Creacion del Vector Assembler con los datos relevantes del Dataset.

En este caso, debido a que todos los datos son relevantes para saber si una persona es propensa a tener un ataque al corazón, dejaremos todos como features para la prediccion del modelo. 

In [8]:
from pyspark.ml.feature import VectorAssembler

cols = ['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs',
        'restecg', 'thalachh', 'exng', 'oldpeak', 
        'slp', 'caa', 'thall']

assembler = VectorAssembler(
    inputCols=cols,
    outputCol="features")

transformed = assembler.transform(df)

#Creamos el dataset final con el vector assembler y los outputs
final_data = transformed.select("features", "output")
final_data.show()

+--------------------+------+
|            features|output|
+--------------------+------+
|[63.0,1.0,3.0,145...|     1|
|[37.0,1.0,2.0,130...|     1|
|[41.0,0.0,1.0,130...|     1|
|[56.0,1.0,1.0,120...|     1|
|[57.0,0.0,0.0,120...|     1|
|[57.0,1.0,0.0,140...|     1|
|[56.0,0.0,1.0,140...|     1|
|[44.0,1.0,1.0,120...|     1|
|[52.0,1.0,2.0,172...|     1|
|[57.0,1.0,2.0,150...|     1|
|[54.0,1.0,0.0,140...|     1|
|[48.0,0.0,2.0,130...|     1|
|[49.0,1.0,1.0,130...|     1|
|[64.0,1.0,3.0,110...|     1|
|[58.0,0.0,3.0,150...|     1|
|[50.0,0.0,2.0,120...|     1|
|[58.0,0.0,2.0,120...|     1|
|[66.0,0.0,3.0,150...|     1|
|[43.0,1.0,0.0,150...|     1|
|[69.0,0.0,3.0,140...|     1|
+--------------------+------+
only showing top 20 rows



In [9]:
final_data.count()

303

Dividimos el dataset en trein y test, con una proporion de 80-20, ya que el dataset al no tener tantas instancias, queremos tener mas informacion para entrenar

In [10]:
# Split en train y test
(df_train, df_test) = final_data.randomSplit([0.8, 0.2], seed=42)

##Creación del modelo

Al querer predecir unos o ceros, para saber si una persona es propensa a sufrir un ataque al corazón, se uso regresion logistica, ya que este modelo nos sirve para clasificar y es sencillo.

In [11]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'features', labelCol = 'output', maxIter=10)
lrModel = lr.fit(df_train)

## Predicción del modelo

In [12]:
prediction_result = lrModel.evaluate(df_test)
prediction_result.predictions.show()

/content/spark-3.2.2-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+------+--------------------+--------------------+----------+
|            features|output|       rawPrediction|         probability|prediction|
+--------------------+------+--------------------+--------------------+----------+
|(13,[0,1,3,4,7,10...|     1|[-1.6024271293504...|[0.16764266337552...|       1.0|
|(13,[0,2,3,4,7,10...|     1|[-4.3409266797788...|[0.01285699774777...|       1.0|
|(13,[0,3,4,6,7,10...|     1|[-3.5230334395098...|[0.02866391713948...|       1.0|
|(13,[0,3,4,7,9,10...|     1|[-0.5150869587615...|[0.37400178283818...|       1.0|
|(13,[0,3,4,7,10,1...|     0|[-2.5446041488268...|[0.07278982082143...|       1.0|
|[35.0,0.0,0.0,138...|     1|[-2.9240991734914...|[0.05097503153331...|       1.0|
|[38.0,1.0,2.0,138...|     1|[0.05966841915935...|[0.51491268055915...|       0.0|
|[39.0,1.0,2.0,140...|     1|[-2.1611473150729...|[0.10329413364568...|       1.0|
|[41.0,1.0,1.0,135...|     1|[-1.6533833752988...|[0.16065220088396...|       1.0|
|[41

## Metricas obtenidas por el modelo

Accuracy del modelo en train

In [13]:
lrModel.summary.accuracy

0.8359375

Accuracy del modelo en test

In [14]:
prediction_result.accuracy

0.851063829787234

Area under ROC del modelo en test

In [15]:
prediction_result.areaUnderROC

0.9176470588235294